In [1]:
import pandas as pd
import numpy as np
import multiprocessing
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import gc
from time import time
import datetime
from tqdm import tqdm_notebook
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.metrics import roc_auc_score
warnings.simplefilter('ignore')
sns.set()
%matplotlib inline

In [2]:
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold,TimeSeriesSplit, GroupKFold
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR

from sklearn.metrics import mean_absolute_error as sk_mean_absolute_error
from sklearn.metrics import roc_auc_score
import numpy as np
import pandas as pd

import eli5
from eli5.sklearn import PermutationImportance

class processutil:
    def _str2class(s):
        if s in globals() and isinstance(globals()[s], type):
                return globals()[s]
        if isinstance(eval(s), type):
            return eval(s)
        if callable(eval(s)):
            return eval(s)
        return None
    

Using TensorFlow backend.


In [3]:
folder_path = '../data/IEEE-CIS-Fraud-Detection/'

In [4]:
df_train = pd.read_pickle(f'{folder_path}/df_train2.gzde', compression='gzip')#.iloc[:100000,:]
# df_test = pd.read_pickle(f'{folder_path}/df_test2.gzde', compression='gzip').iloc[:10000,:]

In [5]:
columns = df_train.columns.tolist()
columns.remove('TransactionID')
columns.remove('TransactionDT')
columns.remove('isFraud')

In [6]:
len(columns)

557

In [7]:
params = {'num_leaves': 491,
          'min_child_weight': 0.03454472573214212,
          'feature_fraction': 0.3797454081646243,
          'bagging_fraction': 0.4181193142567742,
          'min_data_in_leaf': 106,
          'objective': 'binary',
          'max_depth': -1,
          'learning_rate': 0.006883242363721497,
          "boosting_type": "gbdt",
          "bagging_seed": 11,
          "metric": 'auc',
          "verbosity": -1,
          'reg_alpha': 0.3899927210061127,
          'reg_lambda': 0.6485237330340494,
          'random_state': 47
         }

In [8]:
def process(folds, df_train,df_test, columns, is_output_feature_importance=1, verbose=0):

#     aucs = list()
    his = []
    training_start_time = time()
    df_feature_importances_i_list = []
    df_valid_pred = pd.DataFrame()
    df_test_pred = pd.DataFrame()
    if type(df_test) != type(None):
        df_test_pred['TransactionID'] = df_test['TransactionID']
    
    X,y = df_train.sort_values('TransactionDT')[columns], df_train.sort_values('TransactionDT')['isFraud']
    if type(df_test) != type(None):
        X_test = df_test.sort_values('TransactionDT')[columns]
        
    for fold, (trn_idx, test_idx) in enumerate(folds.split(X, y)):
        start_time = time()
        if verbose > 1:
            print('Training on fold {}'.format(fold + 1))
            
        trn_data = lgb.Dataset(X.iloc[trn_idx], label=y.iloc[trn_idx])
        val_data = lgb.Dataset(X.iloc[test_idx], label=y.iloc[test_idx])
        clf = lgb.train(params, trn_data, 10000, valid_sets = [trn_data, val_data], verbose_eval=1000, early_stopping_rounds=500)
        
        y_trn_pred = clf.predict(X.iloc[trn_idx].values)
        y_val_pred = clf.predict(X.iloc[test_idx].values)
        
        original_index = df_train['TransactionID'].values[test_idx]
        df_valid_pred_i = pd.DataFrame({'TransactionID': original_index, 'predict': y_val_pred, 'fold': np.zeros(y_val_pred.shape[0]) + fold})
        df_valid_pred = pd.concat([df_valid_pred, df_valid_pred_i], axis=0)
        
        y_test_pred = None
        if type(df_test)!=type(None):
            y_test_pred = clf.predict(X_test.values)
            df_test_pred_i = pd.DataFrame({fold: y_test_pred})
            df_test_pred = pd.concat([df_test_pred, df_test_pred_i], axis=1)
        
        trn_auc = roc_auc_score(y.iloc[trn_idx].values, y_trn_pred)
        val_auc = roc_auc_score(y.iloc[test_idx].values, y_val_pred)
        
        his.append({'val_auc':val_auc, 'trn_auc':trn_auc, 'y_val_pred':y_val_pred, 'y_test_pred':y_test_pred, 'test_idx':test_idx})
        
        if is_output_feature_importance:
            best_iter = clf.best_iteration
            clf = lgb.LGBMClassifier(**params, num_boost_round=best_iter)
            clf.fit(X.iloc[trn_idx].values, y.iloc[trn_idx].values)
            perm = PermutationImportance(clf, random_state=42).fit(X.iloc[test_idx].values, y.iloc[test_idx].values)
            df_feature_importances_i2 = eli5.explain_weights_dfs(perm, feature_names=columns, top=len(columns))['feature_importances']
            df_feature_importances_i2 = df_feature_importances_i2.sort_values(by=['feature'])
            df_feature_importances_i2 = df_feature_importances_i2.reset_index(drop=True)
            df_feature_importances_i_list.append(df_feature_importances_i2)
        
#         aucs.append(clf.best_score['valid_1']['auc'])
#         his.append({'val_auc':val_auc, 'trn_auc':trn_auc, 'y_val_pred':y_val_pred, 'y_test_pred':y_test_pred, 'test_idx':test_idx})
        if verbose > 0:
            print('Fold {} finished in {}'.format(fold + 1, str(datetime.timedelta(seconds=time() - start_time))))
    
    his = pd.DataFrame(his)
    
    df_feature_importances = None
    if is_output_feature_importance:
        df_feature_importances = df_feature_importances_i_list[0]
        for idx, df_feature_importances_i in enumerate(df_feature_importances_i_list[1:]):
            df_feature_importances = pd.merge(df_feature_importances, df_feature_importances_i, on='feature', suffixes=('', idx + 1))
            
    df_valid_pred = df_valid_pred.sort_values(by=['TransactionID'])
    df_valid_pred = df_valid_pred.reset_index(drop=True)

    if type(df_test) != type(None):
        df_test_pred = df_test_pred.sort_values(by=['TransactionID'])
        df_test_pred = df_test_pred.reset_index(drop=True)
    
    if verbose > 0:
        print('-' * 30)
        print('Training has finished.')
        print('Total training time is {}'.format(str(datetime.timedelta(seconds=time() - training_start_time))))
        print('Mean AUC:', his.val_auc.mean(), his.trn_auc.mean())
        print('-' * 30)
    return his, df_feature_importances, df_valid_pred, df_test_pred, his.val_auc.mean()

In [9]:
folds = TimeSeriesSplit(n_splits=5)

In [10]:
his, df_feature_importances, df_valid_pred, df_test_pred, val_metric = process(folds, df_train,None, columns, is_output_feature_importance=1, verbose=1)

Training until validation scores don't improve for 500 rounds.
[1000]	training's auc: 1	valid_1's auc: 0.907751
Early stopping, best iteration is:
[1022]	training's auc: 1	valid_1's auc: 0.907858
Fold 1 finished in 1:14:35.112509
Training until validation scores don't improve for 500 rounds.
[1000]	training's auc: 0.999999	valid_1's auc: 0.92638
Early stopping, best iteration is:
[819]	training's auc: 0.999985	valid_1's auc: 0.926613
Fold 2 finished in 1:16:09.676091
Training until validation scores don't improve for 500 rounds.
[1000]	training's auc: 0.999973	valid_1's auc: 0.922444
Early stopping, best iteration is:
[1138]	training's auc: 0.999994	valid_1's auc: 0.92272
Fold 3 finished in 1:50:11.975752
Training until validation scores don't improve for 500 rounds.
[1000]	training's auc: 0.999792	valid_1's auc: 0.938852
[2000]	training's auc: 1	valid_1's auc: 0.939422
Early stopping, best iteration is:
[1577]	training's auc: 0.999999	valid_1's auc: 0.939698
Fold 4 finished in 2:58:45

In [11]:
def sort_feature_importances(df_feature_importances, key='average_permutation_weight'):
    df_feature_importances['average_permutation_weight'] = df_feature_importances[
        [col for col in df_feature_importances.columns.tolist() if ('weight' in col) & ('model' not in col)]].mean(
        axis=1)
    df_feature_importances = df_feature_importances.sort_values(by=[key], ascending=False)
    sorted_columns = df_feature_importances.feature.tolist()
    return sorted_columns

In [12]:
sorted_columns = sort_feature_importances(df_feature_importances)

In [13]:
sorted_columns

['PCA_C_1',
 'C1_fq_enc',
 'C13',
 'PCA_C_2',
 'C1',
 'C14',
 'D2',
 'V70',
 'M4_target_mean',
 'uid3_TransactionAmt_mean',
 'addr1_fq_enc',
 'card2_TransactionAmt_mean',
 'addr1__card1',
 'V258',
 'M5',
 'uid3_TransactionAmt_std',
 'uid3_fq_enc',
 'D15_to_std_addr1',
 'D15_to_mean_addr1',
 'C11_fq_enc',
 'card1',
 'C11',
 'card1_TransactionAmt_mean',
 'D2_fq_enc',
 'uid2_TransactionAmt_mean',
 'P_emaildomain__C2',
 'M6',
 'card2_fq_enc',
 'D1',
 'C13_fq_enc',
 'V317',
 'C2',
 'TransactionAmt',
 'addr1',
 'uid2_fq_enc',
 'V294',
 'uid_TransactionAmt_mean',
 'C14_fq_enc',
 'card1_fq_enc',
 'C8',
 'TransactionAmt_log1p',
 'card1_count_full',
 'card3_TransactionAmt_mean',
 'Card_ID',
 'M4',
 'C2_fq_enc',
 'P_emaildomain_fq_enc',
 'ProductCD_target_mean',
 'uid2_TransactionAmt_std',
 'uid_fq_enc',
 'card5__P_emaildomain',
 'card6',
 'uid_TransactionAmt_std',
 'card1_TransactionAmt_std',
 'id_20',
 'card2_count_full',
 'C6_fq_enc',
 'card2_TransactionAmt_std',
 'V87',
 'card2__dist1',
 'car

In [19]:
df_test = pd.read_pickle(f'{folder_path}/df_test2.gzde', compression='gzip')

In [27]:
folds = KFold(n_splits=8, shuffle=False, random_state=42)

In [28]:
his, df_feature_importances, df_valid_pred, df_test_pred, val_metric = process(folds, df_train,df_test, sorted_columns[:200], is_output_feature_importance=0, verbose=1)

Training until validation scores don't improve for 500 rounds.
[1000]	training's auc: 0.998414	valid_1's auc: 0.923991
[2000]	training's auc: 0.99999	valid_1's auc: 0.92559
[3000]	training's auc: 1	valid_1's auc: 0.926309
Early stopping, best iteration is:
[2952]	training's auc: 1	valid_1's auc: 0.926248
Fold 1 finished in 0:08:37.854114
Training until validation scores don't improve for 500 rounds.
[1000]	training's auc: 0.998327	valid_1's auc: 0.939743
[2000]	training's auc: 0.999989	valid_1's auc: 0.942084
[3000]	training's auc: 1	valid_1's auc: 0.942727
Early stopping, best iteration is:
[3072]	training's auc: 1	valid_1's auc: 0.94276
Fold 2 finished in 0:09:03.627249
Training until validation scores don't improve for 500 rounds.
[1000]	training's auc: 0.998543	valid_1's auc: 0.943361
[2000]	training's auc: 0.999994	valid_1's auc: 0.9472
[3000]	training's auc: 1	valid_1's auc: 0.947551
Early stopping, best iteration is:
[2753]	training's auc: 1	valid_1's auc: 0.947671
Fold 3 finish

In [36]:
df_valid_pred.to_pickle(f'{folder_path}/oof_lgbm_train', compression='gzip')

In [37]:
df_test_pred.to_pickle(f'{folder_path}/oof_lgbm_test', compression='gzip')

In [14]:
def add_feature_obo(trial, folds, df_train, columns, trial, max_col = 50):
#     trial = []
    selected_columns = [columns[0]]
    if type(max_col) == type(None):
        max_col = len(columns)
    best_val_metric = 0
    for col in columns[1:]:
        his, df_feature_importances, df_valid_pred, df_test_pred, val_metric = process(folds, df_train,None, selected_columns + [col], is_output_feature_importance=0, verbose=0)
        if val_metric > best_val_metric:
            selected_columns = selected_columns + [col]
            best_val_metric = val_metric
            trial.append({'col_count':len(selected_columns), 'val_metric':val_metric, 'selected_columns':selected_columns})
        if len(selected_columns) > max_col:
            break
    return selected_columns, trial

In [17]:
trial=[]
select_columns, trial = add_feature_obo(trial, folds, df_train, sorted_columns, max_col = 50)